In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import defaultdict


## Data preparation

In [2]:
null = None
def parseReview(fname, maxcount):
    count = 0
    for l in open(fname):
        yield eval(l)
        count += 1
        if count >= maxcount:
            break

def parseData(fname):
    for l in open(fname):
        yield eval(l)
            
def parseDataFromURL(fname):
    for l in urlopen(fname):
        yield eval(l)

In [4]:
business = list(parseData("./business.json"))
review = list(parseReview("./review.json", 120000))
user = list(parseData("./user.json"))

df_business = pd.DataFrame(business)
df_review = pd.DataFrame(review)
df_user = pd.DataFrame(user).drop(columns = ['elite'])

In [5]:
merged = df_review.merge(df_business, how = 'inner', on = 'business_id')
merged_all = merged.merge(df_user, how = 'inner', on = 'user_id')

In [42]:
# merged_all.to_csv('all_data_merged.csv', index = False)

## EDA
- Dataset overview
    - (Done)Count of user: 93921
    - (Done)Count of business: 14766
    - (Done)Count of city: 1204
    - (Done)Count of categories: 155 (There are 1300 in total, we took those with more than 1k stores, which is 155 of them)
- Rating over locations
- Rating over categories 
- Popular categories by city
- Popular words in 1-star and 5-star reviews

In [6]:
# Total user count
len(df_review['user_id'].unique())

93921

In [7]:
# Total business count 
len(df_review['business_id'].unique())

14766

In [8]:
# Total city count
len(df_business['city'].unique())

1204

In [9]:
# Handling catagories
all_cat = {}
def handle_cat(s):
    if s:
        for w in s.split(','):
            if w.strip() in all_cat.keys():
                all_cat[w.strip()] += 1
            else:
                all_cat[w.strip()] = 1
df_business.categories.apply(handle_cat)

# Finding the popular catagory that has more than 1000 business
common_cat = []
for key in all_cat.keys():
    if all_cat[key] >= 1000:
        common_cat.append(key)
print('Done collecting categories')

def classify_cat(s, catagory):
    if s == '':
        return 0
    if s == None:
        return 0
    if catagory in s:
        return 1
    return 0
    
# Adding one hot representation of those catagories
for catagory in common_cat:
    df_business[catagory] = df_business.categories.apply(lambda x: classify_cat(x, catagory))

Done collecting categories


In [10]:
# Total catagories
len(all_cat)

1300

In [11]:
# Common catagories (more than 1k store)
len(common_cat)

155

In [12]:
df_business.groupby('state').agg(np.mean)['stars'].sort_values(ascending = False)

state
NJ     5.000000
TN     5.000000
XWY    4.500000
XGL    4.500000
VT     4.250000
TX     4.166667
CA     4.026316
BAS    4.000000
VA     4.000000
XGM    3.875000
AL     3.833333
GA     3.750000
AZ     3.707185
NV     3.696423
QC     3.635535
WI     3.610691
PA     3.577523
NC     3.542187
OH     3.505341
SC     3.503873
DUR    3.500000
CON    3.500000
CT     3.500000
IL     3.464286
AB     3.385359
ON     3.356504
NY     3.250000
DOW    3.000000
NE     3.000000
AK     2.750000
FL     2.500000
NM     2.500000
WA     2.333333
AR     2.000000
UT     2.000000
BC     1.500000
Name: stars, dtype: float64

In [13]:
merged = df_review.merge(df_business, how = 'inner', on = 'business_id')
merged_all = merged.merge(df_user, how = 'inner', on = 'user_id').dropna()

In [14]:
merged_all.head()

,business_id,cool_x,date,funny_x,review_id,stars_x,text,useful_x,user_id,address,...,compliment_profile,compliment_writer,cool_y,fans,friends,funny_y,name_y,review_count_y,useful_y,yelping_since
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,3100 N Tenaya Way,...,0,0,0,0,frQs7y5qa-X1pvAM0sJe1w,4,Stefan,10,18,2008-07-10 17:24:47
2,ujmEBvifdJM6h6RLv4wQIg,0,2017-02-09 06:45:18,0,8Edv5pKe5lOBoN5UZTUI1w,4.0,My family has used this ER four times in the p...,0,RBXSJA372ilErzNwz0jXvQ,3100 N Tenaya Way,...,0,0,0,0,None,0,Sofia,12,0,2015-08-16 21:26:19
3,ujmEBvifdJM6h6RLv4wQIg,0,2018-08-14 05:25:33,0,kEs5u0cGPtiSQCLW7b0l9g,1.0,I have never been more disappointed by the car...,1,x3brMMbJrAW9PwW5A6YL5w,3100 N Tenaya Way,...,0,0,0,0,None,0,Estela,1,1,2017-12-24 05:30:24
4,ujmEBvifdJM6h6RLv4wQIg,0,2018-05-08 19:27:59,0,Pm6qydbW58BB0zdWCPlBmg,1.0,"Went in for a broken finger, was asked if I wa...",3,Skzdl0sWhW88525a1vr59g,3100 N Tenaya Way,...,0,0,0,0,"ZSgTj4G8hTc-Et1GemnwAg, dr33hGEXuoiFi7TZpTijsQ...",1,Renee,6,2,2015-06-05 03:30:26
5,ujmEBvifdJM6h6RLv4wQIg,0,2014-10-11 06:57:18,0,QWJVYwNxUY_Plu4Ipz33_A,5.0,My mother was at Mountain View for nearly two ...,1,3Y25VDfnQVcuc33T-U3Z6A,3100 N Tenaya Way,...,0,0,0,0,None,0,Yesenia,5,2,2014-10-11 01:53:03


In [146]:
merged_all.to_csv('all_data_merged.csv', index = False)

In [147]:
merged_all.shape

(106804, 198)

In [18]:
# drop open hours for prediction
column = list(merged_all.columns)
column

['business_id',
 'cool_x',
 'date',
 'funny_x',
 'review_id',
 'stars_x',
 'text',
 'useful_x',
 'user_id',
 'address',
 'attributes',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name_x',
 'postal_code',
 'review_count_x',
 'stars_y',
 'state',
 'Active Life',
 'Specialty Food',
 'Restaurants',
 'Food',
 'Chinese',
 'Ethnic Food',
 'Seafood',
 'Sushi Bars',
 'Japanese',
 'Financial Services',
 'Plumbing',
 'Shopping',
 'Local Services',
 'Home Services',
 'Home & Garden',
 'Printing Services',
 'Beauty & Spas',
 'Hair Salons',
 'Hair Stylists',
 'Barbers',
 'Cosmetics & Beauty Supply',
 'Nail Salons',
 'Day Spas',
 'Massage',
 'Professional Services',
 'IT Services & Computer Repair',
 'Breakfast & Brunch',
 'Mexican',
 'Fast Food',
 'Bars',
 'Nightlife',
 'Pubs',
 'Italian',
 'Pizza',
 'Chicken Wings',
 'Bakeries',
 'Fitness & Instruction',
 'Yoga',
 'Event Planning & Services',
 'Trainers',
 'Health & Medical',
 'Physical Therapy',
 'Gyms',
 'Arts & Ent

In [20]:
merged_all_wo_hours = merged_all.drop('hours', axis=1)

In [21]:
merged_all_wo_hours.columns

Index(['business_id', 'cool_x', 'date', 'funny_x', 'review_id', 'stars_x',
       'text', 'useful_x', 'user_id', 'address',
       ...
       'compliment_profile', 'compliment_writer', 'cool_y', 'fans', 'friends',
       'funny_y', 'name_y', 'review_count_y', 'useful_y', 'yelping_since'],
      dtype='object', length=196)